In [2]:
import numpy as np
import scipy.optimize as optimize
from math import exp,sqrt
"""
sh = sheerwood number
d = filament diameter
D_PFBA = Diffusion coefficient of PFAS in water
"""
def Mass_transfer():
    sh = 66
    d = 7e-4
    D_PFBA = 6.2e-10
    k = sh * D_PFBA/d
    return k

k = Mass_transfer()
print("Mass_transfer:",k)

"""
R = Rejection
k = Mass transfer coefficient
Jw = Hydraulic flux
B = Solute(PFBA) Permeability (m/s): 1.12517e-05 (R = 0.3, before modification), 4.82217e-06(R=0.5, after SIMULATED)  3.351e-06 (R = 0.59, after modication), 4.87087e-08 (99% rejection, ideal membrane)
A = Hydraulic Permeability (m/s) : 5.07e-07 (R = 0.3, before modification), 4.4744e-07 (R=0.5, after SIMULATED) 3.889e-07(R=0.59, after modification), 2.9983e-07(99% rejection, ideal membrane)

Test with different membranes assuming a 40% decline in water perm.
Pw1 = 7.38e-07, 7.33e-07, 1.09e-06, 1.23e-06
"""
def func(B):
    R = 95
    k = 2.14e-5   #(m/s)
    Jw = 8.3333e-6  #(m/s, 19LMH)
    w = exp(Jw/k)
    X = (1-R)*Jw
    Z = R* w
    return B - X/Z 

PFBA_Permeabilty =optimize.bisect(func,1e-12,1e-6, xtol=5e-17, rtol= 5e-15, maxiter = 500)
print("PFBA_Permeability:",PFBA_Permeabilty) 


Mass_transfer: 5.845714285714286e-05


ValueError: f(a) and f(b) must have different signs

In [28]:
""" 
Estimating the water permeability coefficient 
"""
k = 5.8457e-5 
jw = 5.2778e-6
Ps = 4.82217e-06 
Ca = 0.001;	P = 0.00007;	K = 0.00060;	
Mg = 0.00035; Na = 0.00593;	SO4 = 0.00089; Cl = 0.00593
Pfba = 0.0001
Cb = (Cl + Na + Mg + Ca + K + SO4 + Pfba)
Pbar = 12
PHI = 1
T = 298.15
def func(Pw):
    z = exp (jw / k)
    cm = ((Ps + jw) * Cb * z) / (jw + Ps * z)
    cp = Ps * Cb * z / (jw + Ps * z)            
    return jw/(Pbar - (PHI * cm - 0.98 * cp) * T *0.083145) - Pw 
Water_Permeabilty =optimize.bisect(func,1e-10,1e-4, xtol=5e-17, rtol= 5e-15, maxiter = 500)
print("Water_Permeabilty:",Water_Permeabilty)

Water_Permeabilty: 4.4743975191252234e-07
